In [1]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

from random import shuffle
# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec



stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text
    #return tokenized
    
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y



In [2]:
doc_stream = stream_docs(path='shuffled_movie_data.csv')
prefix = 'REVIEW'

reviews = []
data_reviews, data_labels = get_minibatch(doc_stream, 50000)
for i in range(50000):
    s = tokenizer(data_reviews[i])
    reviews.append(TaggedDocument(words=s, tags=[prefix + '_%s' % i]))
    #s = data_reviews[i]
    #reviews.append(TaggedDocument(words=utils.to_unicode(s).split(), tags=[prefix + '_%s' % i]))

In [3]:
# MODELO
model = Doc2Vec(min_count=1, window=10, size=300, sample=1e-4, negative=5, workers=7)
model.build_vocab(reviews)
shuffle(reviews)
model.train(reviews,total_examples=model.corpus_count,epochs=150) # puede tomar 25 minutos

663653978

In [4]:
model.save('./model.d2v')